In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 57.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64

In [4]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [5]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [6]:
import random
random.seed(42)
galleries = ['animal','tree']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen


In [7]:
def remove_words(sentence, word):
    return ' '.join([w.replace(word, '') for w in sentence.split()])
    # return ' '.join([w for w in sentence.split() if w != word])
stopwords = ['-dc', 'official', 'App', '이미지' ,'순서','저장','.jpg','.png','.txt','.gif']#['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [8]:
import collections

def remove_top_n_words(data, n):
    words = [word for sentence in data for word in sentence.split(' ')]
    word_count = collections.Counter(words)
    top_n_words = [word for word, _ in word_count.most_common(n)]
    for i in range(len(data)):
        for word in top_n_words:
            if word=='':
                continue
            data[i] = remove_words(data[i], word)
    return data

def remove_empty_sentences(cocktail, real_labels):
  indices = [i for i, sentence in enumerate(cocktail) if sentence.strip() != '']
  cocktail = [cocktail[i] for i in indices]
  real_labels = [real_labels[i] for i in indices]
  return cocktail, real_labels


In [9]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
cocktail = remove_top_n_words(cocktail, 0)
cocktail, real_labels = remove_empty_sentences(cocktail, real_labels)

In [10]:
from transformers import AutoModel, AutoTokenizer
from umap import UMAP
# umap_model = UMAP(random_state=42)
# umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=42)
# model = AutoModel.from_pretrained("klue/roberta-large")
# tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

# model = AutoModel.from_pretrained("klue/bert-base")
# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# from kobert_tokenizer import KoBERTTokenizer
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# import torch
# from transformers import BertModel
# model = BertModel.from_pretrained('skt/kobert-base-v1')

In [11]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  # def __call__(self, target):
  #   return self.tokenizer.tokenize(target)
  def __call__(self, target):
    tokens = self.tokenizer.tokenize(target)
    if len(tokens)==0:
      return []
    if isinstance(tokens[0], str):
      indices = [i for i, token in enumerate(tokens) if not token.startswith('##')]
      tokens = [tokens[i] for i in indices]
    elif isinstance(tokens[0][0], str):
      for j in range(len(tokens)):
        indices = [i for i, token in enumerate(tokens[j]) if not token.startswith('##')]
        tokens[j] = [tokens[j][i] for i in indices]
    return tokens

In [12]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# custom_tokenizer = CustomTokenizer(tokenizer)
# # vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
# vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [13]:
# from umap import UMAP
# from hdbscan import HDBSCAN

# umap_model = UMAP(n_neighbors=5, n_components=5, metric='cosine', low_memory=False)
# hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', prediction_data=True)

In [14]:
from bertopic.vectorizers import ClassTfidfTransformer
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
bertopic_model_embed_token = BERTopic(language='multilingual',
                          nr_topics=100,
                          # ctfidf_model=ctfidf_model,
                          top_n_words=1)
# bertopic_model_embed_token = BERTopic(embedding_model=model,
#                                       vectorizer_model=vectorizer,
#                                       ctfidf_model=ctfidf_model,
#                                       nr_topics=100,
#                                       top_n_words=1,
#                                       calculate_probabilities=False)
topics, _ = bertopic_model_embed_token.fit_transform(cocktail)

In [15]:
topic_info = bertopic_model_embed_token.get_topic_info()
number_of_topics = len(topic_info) - 1
print(f"생성된 주제의 수: {number_of_topics}")
lst = []
for index, row in topic_info.iterrows():
    topic_num = row['Topic']
    if topic_num != -1:
        topic_name = row['Name']
        pure_topic_name = "_".join(topic_name.split("_")[1:])
        lst.append(pure_topic_name)
for i in lst:
  print(i)

생성된 주제의 수: 99
로보쿤
햄스터
꽃이
신엽
물을
데로닉스
이거
잎이
가격은
쥐친구
흑설기와
날씨
병원
흙에
식물
터널
거미줄
뿌리가
죽음
토마토
이름이
이름이
잠자는쥐
냄새
쥐가
고양이
핸들링
식물등
귀엽다
사료
나무
플로리다
쳇바퀴
맛있어보임
우리집
화장실
모래
서울
떨리는
프롬프트
코코넛
식물
베르디
비둘기
알로카시아
케이지
유혈목이
기니피그
축산
분양
일베
빨리
수컷이
원숭이
비슷
금방
올리브
해주세요
딸기
노견
냉장고를
얼음
블루베리
고양이
올해
강아지
플라스틱
유튜브
나무위키
곰스터가
잠만
ㅇㅇ
돼지
피그미다람쥐
끓인
개구리
유통기한
생태계교란
철물점
상어
거북이
버섯
식쇼
고사리입니다
제라늄
햄스터
벌레인지
햇빛아래
아이스크림
눈이
김나나
멍청햄
된다는것만
코코봉
전면부를
크기
뚱땡
코코넛
이렇게라도


In [16]:
table = [[0 for r in range(2)] for t in range(number_of_topics)]

for j in range(number_of_topics):
  for i in range(len(topics)):
    if topics[i]==j:
      table[j][real_labels[i]]+=1
print(table)

[[926, 679], [1352, 0], [7, 540], [91, 274], [27, 327], [119, 110], [63, 138], [2, 188], [61, 120], [150, 4], [117, 27], [46, 72], [105, 12], [4, 93], [0, 95], [48, 38], [20, 62], [1, 81], [29, 47], [5, 69], [10, 63], [0, 72], [68, 3], [39, 31], [65, 1], [52, 13], [58, 6], [0, 64], [46, 16], [53, 9], [5, 57], [16, 45], [53, 8], [40, 15], [43, 9], [46, 5], [43, 3], [14, 31], [17, 24], [33, 6], [36, 2], [2, 32], [1, 32], [31, 2], [1, 31], [31, 1], [29, 1], [28, 1], [29, 0], [14, 14], [26, 0], [1, 25], [24, 2], [25, 0], [2, 23], [0, 21], [0, 20], [8, 12], [4, 16], [18, 1], [19, 0], [0, 18], [4, 14], [16, 1], [2, 15], [14, 3], [6, 11], [10, 6], [14, 2], [15, 1], [16, 0], [14, 1], [14, 1], [15, 0], [1, 14], [10, 5], [4, 11], [14, 1], [5, 10], [12, 3], [12, 2], [0, 14], [0, 13], [0, 13], [0, 13], [13, 0], [2, 10], [1, 11], [4, 8], [9, 3], [11, 1], [10, 1], [10, 1], [3, 8], [1, 10], [6, 5], [10, 0], [10, 0], [0, 10]]


In [17]:
### Reducing outliers
topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="c-tf-idf", threshold=0.1)
topics = bertopic_model_embed_token.reduce_outliers(cocktail, topics, strategy="distributions")

In [18]:
hierarchical_topics = bertopic_model_embed_token.hierarchical_topics(cocktail)

100%|██████████| 98/98 [00:00<00:00, 115.29it/s]


In [19]:
# Hierarchical topics
# linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
# hierarchical_topics = bertopic_model_embed_token.hierarchical_topics(cocktail, linkage_function=linkage_function)

In [20]:
bertopic_model_embed_token.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [21]:
hierarchical_topics

,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
97,196,햄스터,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",194,꽃이,195,햄스터,1.480643
96,195,햄스터,"[1, 5, 9, 10, 11, 12, 15, 16, 18, 19, 22, 23, ...",193,너무,186,햄스터,1.270586
95,194,꽃이,"[0, 2, 3, 4, 6, 7, 8, 13, 14, 17, 20, 21, 30, ...",121,이름이,166,꽃이,1.261965
94,193,너무,"[5, 9, 10, 11, 15, 16, 18, 19, 22, 24, 25, 27,...",192,고양이,191,너무,1.193425
93,192,고양이,"[25, 40, 63, 97]",103,고양이,102,코코넛,1.163461
...,...,...,...,...,...,...,...,...
4,103,고양이,"[25, 63]",63,고양이,25,고양이,0.745362
3,102,코코넛,"[40, 97]",40,코코넛,97,코코넛,0.732061
2,101,물을,"[4, 7]",7,잎이,4,물을,0.720600
1,100,꽃이,"[2, 3]",3,신엽,2,꽃이,0.690308


In [22]:
bertopic_model_embed_token.visualize_heatmap()

In [23]:
tree = bertopic_model_embed_token.get_topic_tree(hierarchical_topics)

In [24]:
print(tree)

.
├─꽃이
│    ├─이름이
│    │    ├─■──나무 ── Topic: 30
│    │    └─이름이
│    │         ├─■──이름이 ── Topic: 21
│    │         └─■──이름이 ── Topic: 20
│    └─꽃이
│         ├─잎이
│         │    ├─■──베르디 ── Topic: 42
│         │    └─물을
│         │         ├─잎이
│         │         │    ├─■──뿌리가 ── Topic: 17
│         │         │    └─물을
│         │         │         ├─물을
│         │         │         │    ├─■──잎이 ── Topic: 7
│         │         │         │    └─■──물을 ── Topic: 4
│         │         │         └─■──식물 ── Topic: 14
│         │         └─■──흙에 ── Topic: 13
│         └─꽃이
│              ├─꽃이
│              │    ├─■──가격은 ── Topic: 8
│              │    └─꽃이
│              │         ├─꽃이
│              │         │    ├─■──로보쿤 ── Topic: 0
│              │         │    └─꽃이
│              │         │         ├─■──신엽 ── Topic: 3
│              │         │         └─■──꽃이 ── Topic: 2
│              │         └─■──이거 ── Topic: 6
│              └─■──금방 ── Topic: 55
└─햄스터
     ├─너무
     │    ├─고양이


In [25]:
df = hierarchical_topics

selected_topics_ids = set()

for index, row in df[df['Distance'] >= 1].iterrows():
    left_child = df[df['Parent_ID'] == row['Child_Left_ID']]
    if not left_child.empty and left_child.iloc[0]['Distance'] < 1:
        selected_topics_ids.add(row['Child_Left_ID'])

    right_child = df[df['Parent_ID'] == row['Child_Right_ID']]
    if not right_child.empty and right_child.iloc[0]['Distance'] < 1:
        selected_topics_ids.add(row['Child_Right_ID'])
selected_topics_ids = list(selected_topics_ids)
print(selected_topics_ids)

['163', '102', '125', '120', '168', '160', '141', '118', '112', '154', '115', '149', '135', '126', '103', '121', '158', '164', '147', '138', '165', '167', '145', '161', '130', '122', '166', '139']


In [26]:
selected_topic_names=[]
for topic_id in selected_topics_ids:
    topic_name = df[df['Parent_ID'] == topic_id]['Parent_Name'].iloc[0]
    selected_topic_names.append(topic_name)
    print(f"ID: {topic_id}, Name: {topic_name}")

ID: 163, Name: 너무
ID: 102, Name: 코코넛
ID: 125, Name: 식쇼
ID: 120, Name: 프롬프트
ID: 168, Name: 올리브
ID: 160, Name: 생태계교란
ID: 141, Name: 햇빛
ID: 118, Name: 돼지
ID: 112, Name: 쳇바퀴
ID: 154, Name: 블루베리
ID: 115, Name: 쥐친구
ID: 149, Name: 피그미다람쥐
ID: 135, Name: 햄스터
ID: 126, Name: 유혈목이
ID: 103, Name: 고양이
ID: 121, Name: 이름이
ID: 158, Name: 잠자는쥐
ID: 164, Name: 이거
ID: 147, Name: 토마토
ID: 138, Name: 우리집
ID: 165, Name: 알로카시아
ID: 167, Name: 흑설기와
ID: 145, Name: 곰스터
ID: 161, Name: 나무위키
ID: 130, Name: 일본
ID: 122, Name: 냄새
ID: 166, Name: 꽃이
ID: 139, Name: 케이지


In [27]:
!pip install fasttext
import fasttext
from sklearn.cluster import KMeans
import numpy as np
from gensim import models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199777 sha256=74185f6081f980aaddcdc1c0865783cdf07c8fbce0b1aad2003b8eb31c29a544
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [28]:
model_path = '/content/drive/MyDrive/jolnon/cc.ko.100.bin'

m_fasttext = models.fasttext.load_facebook_model(model_path)

ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa1\x9c'; replacing invalid characters, using '\\xed\\xa1\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb0\x80'; replacing invalid characters, using '\\xed\\xb0\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa7\x80'; replacing invalid characters, using '\\xed\\xa7\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb8\xb0'; replacing invalid characters, using '\\xed\\xb8\\xb0'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa0\x9c'; replacing invalid characters, using '\\xed\\xa0\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa6\xac'; replacing invalid characters, using '\\xed\\xa6\\xac'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb3\xb5'; replacing invalid characters, using '\\xed\\xb

In [29]:
embeddings = []

# for word in selected_topic_names:
#     # embeddings.append(word2vec[word])
#     if word in word2vec.keys():
#         embedding = word2vec[word]
#         embeddings.append(embedding)
#     else:
#         embeddings.append(np.random.randn(300))

embeddings = [m_fasttext.wv[selected_topic_names[i]] for i in range(len(selected_topic_names))]
embed_ids = [selected_topics_ids[i] for i in range(len(selected_topic_names))]

embeddings = np.array(embeddings)

n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(embeddings)

clustered_topics = [[],[],[]]
clustered_topics_name = [[], [], []]

# 결과 출력
for i, label in enumerate(kmeans.labels_):
    clustered_topics[label].append(embed_ids[i])
    clustered_topics_name[label].append(selected_topic_names[i])
    # print(f"Topic: {selected_topic_names[i]}, Cluster: {label}")

for r in clustered_topics_name:
    print(r)

['햇빛', '돼지', '냄새', '꽃이']
['너무', '코코넛', '식쇼', '프롬프트', '올리브', '생태계교란', '쳇바퀴', '블루베리', '쥐친구', '피그미다람쥐', '햄스터', '유혈목이', '고양이', '이름이', '잠자는쥐', '이거', '토마토', '우리집', '알로카시아', '흑설기와', '곰스터', '나무위키', '일본', '케이지']
[]


In [30]:
post_counts = [0 for i in range(number_of_topics)]
for i in range(len(post_counts)):
    post_counts[i] = topics.count(i)

cluster_sums = []

for cluster in clustered_topics:
    total_sum = 0
    for parent_id in cluster:
        row = hierarchical_topics[hierarchical_topics['Parent_ID'] == parent_id].iloc[0]
        topic_ids = row['Topics']
        cluster_sum = sum(post_counts[topic_id] for topic_id in topic_ids)
        total_sum += cluster_sum
    cluster_sums.append(total_sum)

for idx, cluster_sum in enumerate(cluster_sums):
    print(f"Cluster {idx} contains {cluster_sum} documents.")


Cluster 0 contains 8644 documents.
Cluster 1 contains 10408 documents.
Cluster 2 contains 0 documents.
